In [49]:
# import os
import pandas as pd
import numpy as np

# 데이터 Load

In [50]:
main = pd.read_csv("BDP-Project/gas_data/main.csv", encoding="cp949")
oil = pd.read_csv("BDP-Project/gas_data/oil.csv", encoding="cp949")
electricity = pd.read_csv("BDP-Project/gas_data/electricity.csv", encoding="cp949")
import_index = pd.read_csv("BDP-Project/gas_data/import_index.csv", encoding="cp949")
lng = pd.read_csv("BDP-Project/gas_data/lng.csv", encoding="cp949")
introduction = pd.read_csv("BDP-Project/gas_data/introduction.csv", encoding="utf-8")
temp2013 = pd.read_csv("BDP-Project/gas_data/temp2013.csv", encoding="cp949")
temp2014 = pd.read_csv("BDP-Project/gas_data/temp2014.csv", encoding="cp949")
temp2015 = pd.read_csv("BDP-Project/gas_data/temp2015.csv", encoding="cp949")
temp2016 = pd.read_csv("BDP-Project/gas_data/temp2016.csv", encoding="cp949")
temp2017 = pd.read_csv("BDP-Project/gas_data/temp2017.csv", encoding="cp949")
temp2018 = pd.read_csv("BDP-Project/gas_data/temp2018.csv", encoding="cp949")

# Main data preprocessing

In [51]:
main['구분'].unique()

array(['A', 'B', 'C', 'D', 'E', 'G', 'H'], dtype=object)

In [52]:
#main의 구분
d_map={}
for i, d in enumerate(main['구분'].unique()):
  d_map[d] = i

#A,B,C,D,E,F,..를 0,1,2,3,4,5,6으로 구분
main['구분'] = main['구분'].map(d_map)
main['구분']

0         0
1         0
2         0
3         0
4         0
         ..
368083    6
368084    6
368085    6
368086    6
368087    6
Name: 구분, Length: 368088, dtype: int64

In [53]:
main['연월일'] = pd.to_datetime(main['연월일'])
main['year'] = main['연월일'].dt.year
main['month'] = main['연월일'].dt.month
main['day'] = main['연월일'].dt.day

In [54]:
def outliers_processing(df):
  q1, q3 = np.percentile(df, [25, 75])
  iqr = q3-q1
  lower_bound = q1-(iqr*1.5)
  upper_bound = q3 + (iqr*1.5)
  return lower_bound, upper_bound

In [55]:
main_by_sections = {}
for i in range(7):
  main_by_sections[i] = main[main['구분'] == i]
main_by_sections

{0:               연월일  시간  구분       공급량  year  month  day
 0      2013-01-01   1   0  2497.129  2013      1    1
 1      2013-01-01   2   0  2363.265  2013      1    1
 2      2013-01-01   3   0  2258.505  2013      1    1
 3      2013-01-01   4   0  2243.969  2013      1    1
 4      2013-01-01   5   0  2344.105  2013      1    1
 ...           ...  ..  ..       ...   ...    ...  ...
 315523 2018-12-31  20   0  3040.868  2018     12   31
 315524 2018-12-31  21   0  2973.577  2018     12   31
 315525 2018-12-31  22   0  2840.272  2018     12   31
 315526 2018-12-31  23   0  2692.385  2018     12   31
 315527 2018-12-31  24   0  2583.340  2018     12   31
 
 [52584 rows x 7 columns],
 1:               연월일  시간  구분       공급량  year  month  day
 8760   2013-01-01   1   1  2169.093  2013      1    1
 8761   2013-01-01   2   1  2051.733  2013      1    1
 8762   2013-01-01   3   1  1923.253  2013      1    1
 8763   2013-01-01   4   1  1927.253  2013      1    1
 8764   2013-01-01   5   1  20

In [56]:
lower = {}
upper = {}
for i in range(7):
  lower[i] = {}
  upper[i] = {}
  for j in range(12):
    lower[i][j], upper[i][j]= outliers_processing(main_by_sections[i][main_by_sections[i]['month'] == j+1]['공급량'])



In [57]:
main['lower'] =""
main['upper'] =""

In [58]:
for i in range(7):
  for j in range(12):
    main.loc[(main['구분']==i)&(main['month']==j+1),'lower' ] = lower[i][j]
    main.loc[(main['구분']==i)&(main['month']==j+1),'upper' ] = upper[i][j]

In [59]:
main['이상치처리_공급량'] =""

In [60]:
main['이상치처리_공급량'] = main.apply(lambda x: x['lower'] if x['공급량'] < x['lower'] else x['upper'] if x['공급량'] > x['upper'] else x['공급량'], axis = 1)

In [61]:
main['공급량'] = main['이상치처리_공급량']
main = main.drop(['lower', 'upper', '이상치처리_공급량'], axis = 1)
main

,연월일,시간,구분,공급량,year,month,day
0,2013-01-01,1,0,2497.129,2013,1,1
1,2013-01-01,2,0,2363.265,2013,1,1
2,2013-01-01,3,0,2258.505,2013,1,1
3,2013-01-01,4,0,2243.969,2013,1,1
4,2013-01-01,5,0,2344.105,2013,1,1
...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31
368084,2018-12-31,21,6,669.961,2018,12,31
368085,2018-12-31,22,6,657.941,2018,12,31
368086,2018-12-31,23,6,610.953,2018,12,31


# Electricity data preprocessing

In [62]:
electricity['연월'] = pd.to_datetime(electricity['연월'])
electricity['year'] = electricity['연월'].dt.year
electricity['month'] = electricity['연월'].dt.month

In [63]:
electricity = electricity.drop(['연월'], axis = 1)
electricity

,가스가격,전기가격,상대가격,year,month
0,88.14,104.50,84.34,2007,1
1,88.14,104.50,84.34,2007,2
2,86.15,104.50,82.44,2007,3
3,86.15,104.50,82.44,2007,4
4,89.45,104.50,85.60,2007,5
...,...,...,...,...,...
184,105.02,111.45,94.23,2022,5
185,105.02,111.45,94.23,2022,6
186,112.03,102.00,109.83,2022,7
187,112.11,102.00,109.91,2022,8


In [64]:
electricity_join = pd.merge(main,electricity, how='inner',on=('year', 'month'))

In [65]:
electricity = electricity_join

# Oil data preprocessing

In [66]:
oil['연월'] = pd.to_datetime(oil['연월'])
oil['year'] = oil['연월'].dt.year
oil['month'] = oil['연월'].dt.month

In [67]:
oil = oil.drop(['연월'], axis = 1)
oil

,가스가격,기름가격,상대가격,year,month
0,73.60,97.08,0.76,2007,1
1,73.60,97.25,0.76,2007,2
2,71.55,100.61,0.72,2007,3
3,71.55,103.96,0.69,2007,4
4,74.89,107.27,0.70,2007,5
...,...,...,...,...,...
184,121.11,219.94,0.56,2022,5
185,121.52,231.03,0.53,2022,6
186,134.92,221.45,0.61,2022,7
187,173.16,200.63,0.87,2022,8


In [68]:
oil_join = pd.merge(main,oil, how='inner',on=('year', 'month'))

In [69]:
oil = oil_join

# Temp data preprocessing

In [70]:
temp_mean2013 = pd.pivot_table(temp2013, values="평균기온(℃)", index=["날짜"], aggfunc = np.mean).reset_index()
temp_mean2014 = pd.pivot_table(temp2014, values="평균기온(℃)", index=["날짜"], aggfunc = np.mean).reset_index()
temp_mean2015 = pd.pivot_table(temp2015, values="평균기온(℃)", index=["날짜"], aggfunc = np.mean).reset_index()
temp_mean2016 = pd.pivot_table(temp2016, values="평균기온(℃)", index=["날짜"], aggfunc = np.mean).reset_index()
temp_mean2017 = pd.pivot_table(temp2017, values="평균기온(℃)", index=["날짜"], aggfunc = np.mean).reset_index()
temp_mean2018 = pd.pivot_table(temp2018, values="평균기온(℃)", index=["날짜"], aggfunc = np.mean).reset_index()

In [71]:
temp_mean2013['날짜'] = temp_mean2013['날짜'].str.slice(1,)
temp_mean2014['날짜'] = temp_mean2014['날짜'].str.slice(1,)
temp_mean2015['날짜'] = temp_mean2015['날짜'].str.slice(1,)
temp_mean2016['날짜'] = temp_mean2016['날짜'].str.slice(1,)
temp_mean2017['날짜'] = temp_mean2017['날짜'].str.slice(1,)
temp_mean2018['날짜'] = temp_mean2018['날짜'].str.slice(1,)

In [72]:
def df_duplicate(df):
  new_df = pd.DataFrame()
  for _ in range(7):
    new_df = pd.concat([new_df, df])
  return new_df

In [73]:
weather_2013_total = df_duplicate(temp_mean2013)
weather_2014_total = df_duplicate(temp_mean2014)
weather_2015_total = df_duplicate(temp_mean2015)
weather_2016_total = df_duplicate(temp_mean2016)
weather_2017_total = df_duplicate(temp_mean2017)
weather_2018_total = df_duplicate(temp_mean2018)

In [74]:
weather_total = pd.concat([weather_2013_total, weather_2014_total, weather_2015_total, 
                           weather_2016_total, weather_2017_total, weather_2018_total])

In [75]:
weather_total

,날짜,평균기온(℃)
0,2013-01-01,-4.7
1,2013-01-02,-11.7
2,2013-01-03,-13.2
3,2013-01-04,-10.7
4,2013-01-05,-7.0
...,...,...
360,2018-12-27,-10.1
361,2018-12-28,-10.5
362,2018-12-29,-7.6
363,2018-12-30,-6.8


In [76]:
weather_total['날짜'] = pd.to_datetime(weather_total['날짜'])
weather_total['year'] = weather_total['날짜'].dt.year 
weather_total['month'] = weather_total['날짜'].dt.month 
weather_total['day'] = weather_total['날짜'].dt.day

In [77]:
weather_total.drop(['날짜'], axis = 1)

,평균기온(℃),year,month,day
0,-4.7,2013,1,1
1,-11.7,2013,1,2
2,-13.2,2013,1,3
3,-10.7,2013,1,4
4,-7.0,2013,1,5
...,...,...,...,...
360,-10.1,2018,12,27
361,-10.5,2018,12,28
362,-7.6,2018,12,29
363,-6.8,2018,12,30


# Import data preprocessing

In [78]:
#  '가스'가 포함된 행 추출
import_index = import_index[import_index['계정코드별'].str.contains('가스', na = False)]

In [79]:
import_index.drop(['Unnamed: 76','단위','항목'], axis=1, inplace=True)

In [80]:
# 달러기준만 추출
import_index = import_index[import_index['통화계약구분코드별'].str.contains('달러기준', na = False)]

In [81]:
import_index.drop(['통화계약구분코드별'], axis=1, inplace=True)
import_index.head()

,계정코드별,2013.01 월,2013.02 월,2013.03 월,2013.04 월,2013.05 월,2013.06 월,2013.07 월,2013.08 월,2013.09 월,...,2018.03 월,2018.04 월,2018.05 월,2018.06 월,2018.07 월,2018.08 월,2018.09 월,2018.10 월,2018.11 월,2018.12 월
56,"석탄,원유및천연가스",190.46,194.75,187.86,182.29,180.80,180.65,183.44,189.30,190.54,...,118.77,126.23,134.87,134.72,133.05,132.79,140.82,144.15,127.28,116.86
413,천연가스(LNG),151.82,154.08,154.76,151.69,151.11,152.35,150.72,157.01,154.67,...,93.73,97.29,101.17,101.01,100.97,105.19,111.67,113.75,115.46,115.23
416,원유및천연가스,196.33,201.51,193.62,187.19,185.20,185.27,189.62,196.52,197.51,...,113.67,122.51,132.32,131.13,130.40,130.57,139.00,142.63,123.17,111.28
734,액화석유가스,223.27,212.75,209.24,190.31,176.83,177.29,185.24,191.71,201.85,...,110.71,110.57,117.45,130.96,131.33,137.18,143.90,153.17,124.75,100.99


In [82]:
import_index = import_index.transpose() #행 열 전환
import_index.rename(columns=import_index.iloc[0], inplace=True) # 행열이 전환된 데이터프레임의 열 이름 제대로 수정
import_index = import_index.drop(import_index.index[0])

In [83]:
import_index['연월일'] = import_index.index

In [84]:
import_index = import_index.reset_index(drop=True)

In [85]:
import_index

,"석탄,원유및천연가스",천연가스(LNG),원유및천연가스,액화석유가스,연월일
0,190.46,151.82,196.33,223.27,2013.01 월
1,194.75,154.08,201.51,212.75,2013.02 월
2,187.86,154.76,193.62,209.24,2013.03 월
3,182.29,151.69,187.19,190.31,2013.04 월
4,180.8,151.11,185.2,176.83,2013.05 월
...,...,...,...,...,...
67,132.79,105.19,130.57,137.18,2018.08 월
68,140.82,111.67,139.0,143.9,2018.09 월
69,144.15,113.75,142.63,153.17,2018.10 월
70,127.28,115.46,123.17,124.75,2018.11 월


In [86]:
import_index['연월일'] = import_index['연월일'].str.slice(0,7)

In [87]:
import_index

,"석탄,원유및천연가스",천연가스(LNG),원유및천연가스,액화석유가스,연월일
0,190.46,151.82,196.33,223.27,2013.01
1,194.75,154.08,201.51,212.75,2013.02
2,187.86,154.76,193.62,209.24,2013.03
3,182.29,151.69,187.19,190.31,2013.04
4,180.8,151.11,185.2,176.83,2013.05
...,...,...,...,...,...
67,132.79,105.19,130.57,137.18,2018.08
68,140.82,111.67,139.0,143.9,2018.09
69,144.15,113.75,142.63,153.17,2018.10
70,127.28,115.46,123.17,124.75,2018.11


In [88]:
for i in range(len(import_index['연월일'])):
    import_index['연월일'][i] = import_index["연월일"][i].replace('.', '-')

In [89]:
import_index

,"석탄,원유및천연가스",천연가스(LNG),원유및천연가스,액화석유가스,연월일
0,190.46,151.82,196.33,223.27,2013-01
1,194.75,154.08,201.51,212.75,2013-02
2,187.86,154.76,193.62,209.24,2013-03
3,182.29,151.69,187.19,190.31,2013-04
4,180.8,151.11,185.2,176.83,2013-05
...,...,...,...,...,...
67,132.79,105.19,130.57,137.18,2018-08
68,140.82,111.67,139.0,143.9,2018-09
69,144.15,113.75,142.63,153.17,2018-10
70,127.28,115.46,123.17,124.75,2018-11


In [90]:
import_index['연월일'] = pd.to_datetime(import_index['연월일'])
import_index['year'] = import_index['연월일'].dt.year
import_index['month'] = import_index['연월일'].dt.month

In [91]:
import_index.drop(['연월일'], axis = 1, inplace = True)

In [92]:
import_index

,"석탄,원유및천연가스",천연가스(LNG),원유및천연가스,액화석유가스,year,month
0,190.46,151.82,196.33,223.27,2013,1
1,194.75,154.08,201.51,212.75,2013,2
2,187.86,154.76,193.62,209.24,2013,3
3,182.29,151.69,187.19,190.31,2013,4
4,180.8,151.11,185.2,176.83,2013,5
...,...,...,...,...,...,...
67,132.79,105.19,130.57,137.18,2018,8
68,140.82,111.67,139.0,143.9,2018,9
69,144.15,113.75,142.63,153.17,2018,10
70,127.28,115.46,123.17,124.75,2018,11


# 전처리 csv 저장

In [94]:
main.to_csv("BDP-Project/pre_main.csv")
import_index.to_csv("BDP-Project/pre_import_index.csv")
introduction.to_csv("BDP-Project/pre_introduction.csv")
weather_total.to_csv("BDP-Project/pre_weather_total.csv")
electricity.to_csv("BDP-Project/pre_electricity.csv")
oil.to_csv("BDP-Project/pre_oil.csv")

# 전체 데이터 merge

In [45]:
main_join = pd.merge(main, import_index, how="inner", on=("year", "month"))

In [46]:
main_join = pd.merge(main, introduction, how="inner", on=("year"))

In [47]:
main_join = pd.merge(main_join, weather_total, how="inner", on=("year", "month", "day"))

In [93]:
# main_join = pd.merge(main_join, electricity, how="inner", on=("year", "month"))

In [ ]:
main_join = pd.merge(main_join, oil, how="inner", on=("year", "month"))

# 전처리 데이터 저장

In [51]:
main_join.to_csv("BDP-Project/gas_data/preprocessing.csv")

# test train data split

In [ ]:
from sklearn.model_selection import train_test_split

target = main['공급량']
main = main.drop(['공급량'], axis = 1)
data = main

x_train, x_text, y_train, y_test = train_test_split(data, target, test_size=0.3, shuffle=True, random_state=34)